<a href="https://colab.research.google.com/github/Ali-Alameer/AI_fairness/blob/main/disparate_impact_remover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook demonstrates the ability of the DisparateImpactRemover algorithm.
The algorithm corrects for imbalanced selection rates between unprivileged and privileged groups at various levels of repair. It follows the guidelines set forth by [1] for training the algorithm and classifier and uses the AdultDataset as an example.

In [ ]:
!pip install 'aif360[all]'
!wget https://raw.githubusercontent.com/Ali-Alameer/AI_fairness/main/common_utils.py

In [ ]:
import urllib.request 
# For Adult dataset
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",'/usr/local/lib/python3.9/dist-packages/aif360/data/raw/adult/adult.data')   
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",'/usr/local/lib/python3.9/dist-packages/aif360/data/raw/adult/adult.test')  
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",'/usr/local/lib/python3.9/dist-packages/aif360/data/raw/adult/adult.names') 


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import matplotlib.pyplot as plt

import sys
sys.path.append("../")
import warnings

import numpy as np
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as SVM
from sklearn.preprocessing import MinMaxScaler

from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from common_utils import compute_metrics

In [4]:
protected = 'sex'
ad = AdultDataset(protected_attribute_names=[protected],
    privileged_classes=[['Male']], categorical_features=[],
    features_to_keep=['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'])

In [5]:
scaler = MinMaxScaler(copy=False)

In [6]:
test, train = ad.split([16281])
train.features = scaler.fit_transform(train.features)
test.features = scaler.fit_transform(test.features)

index = train.feature_names.index(protected)

The repair_level parameter in DisparateImpactRemover specifies the amount or intensity of the repair applied to the data. It is typically a value between 0 and 1, where 0 means no repair (i.e., no modification of the data) and 1 means maximum repair (i.e., full modification of the data to eliminate disparate impact). The value of repair_level determines the trade-off between fairness and accuracy in the resulting model. Higher values of repair_level may result in more fairness but potentially lower accuracy, while lower values may result in higher accuracy but less fairness.

In [ ]:
acc = []
dis_impact = []
ave_odds_diff = []
Statistical_parity_difference = []
Equal_opportunity_difference = []
Theil_index = []
repair_level_all = []

for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train)
    test_repd = di.fit_transform(test)
    
    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    lmod = LogisticRegression(class_weight='balanced', solver='liblinear')
    lmod.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = lmod.predict(X_te)

    p = [{protected: 1}]
    u = [{protected: 0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    print("Repair Level = %f" % level)
    dis_impact.append(cm.disparate_impact())
    metric_test = compute_metrics(test_repd, test_repd_pred, 
                                      unprivileged_groups=u, privileged_groups=p,
                                      disp = True)
    acc.append(metric_test["Balanced accuracy"])
    ave_odds_diff.append(metric_test["Average odds difference"])
    Statistical_parity_difference.append(metric_test["Statistical parity difference"])
    Equal_opportunity_difference.append(metric_test["Equal opportunity difference"])
    Theil_index.append(metric_test["Theil index"])
    repair_level_all.append(level)



These fairness metrics provide quantitative measures to assess the presence of bias and fairness in machine learning systems, and can be used to evaluate the performance of fairness-aware machine learning algorithms and techniques.

- Statistical Parity Difference: This metric measures the difference in the rate of favorable outcomes between the unprivileged and privileged groups. The ideal value is 0, and fairness is indicated by values between -0.1 and 0.1.

- Disparate Impact: This metric computes the ratio of the rate of favorable outcomes between the unprivileged and privileged groups. The ideal value is 1.0, and fairness is indicated by values between 0.8 and 1.25. A value < 1 implies higher benefit for the privileged group, while a value > 1 implies higher benefit for the unprivileged group.

- Average Odds Difference: This metric measures the average difference in false positive rate and true positive rate between the unprivileged and privileged groups. The ideal value is 0, and fairness is indicated by values between -0.1 and 0.1. A value < 0 implies higher benefit for the privileged group, while a value > 0 implies higher benefit for the unprivileged group.

- Equal Opportunity Difference: This metric computes the difference in true positive rates between the unprivileged and privileged groups. The ideal value is 0, and fairness is indicated by values between -0.1 and 0.1. A value < 0 implies higher benefit for the privileged group, while a value > 0 implies higher benefit for the unprivileged group.

- Theil Index: This metric measures the inequality in benefit allocation for individuals, computed as the generalized entropy of benefit with alpha = 1. The ideal value is 0, with lower scores indicating better fairness and higher scores indicating more problematic fairness.

In [ ]:
# Create figure and axes
fig, ax1 = plt.subplots()

# Plotting ave_odds_diff against repair_level_all on the left-hand y-axis
ax1.plot(repair_level_all, ave_odds_diff, label='Average Odds Difference')
ax1.plot(repair_level_all, Statistical_parity_difference, label='Statistical Parity Difference')
ax1.plot(repair_level_all, Equal_opportunity_difference, label='Equal Opportunity Difference')
ax1.plot(repair_level_all, Theil_index, label='Theil Index')
ax1.plot(repair_level_all, dis_impact, label='Disparate Impact')
ax1.set_xlabel('Repair Level')
ax1.set_ylabel('Metrics')

# Create a twin axes sharing the x-axis with ax1
ax2 = ax1.twinx()

# Plotting acc against repair_level_all on the right-hand y-axis
ax2.plot(repair_level_all, acc, label='Accuracy')
ax2.set_ylabel('Accuracy')

# Add legend
lines = ax1.get_lines() + ax2.get_lines()
ax1.legend(lines, [line.get_label() for line in lines])

# Show the plot
plt.show()

After correcting for disparate impact, we observe that the most improved metric on the graph was actually the disparate impact

In [ ]:
%matplotlib inline
plt.plot(np.linspace(0, 1, 11), dis_impact, marker='o')
plt.plot([0, 1], [1, 1], 'g')
plt.plot([0, 1], [0.8, 0.8], 'r')
plt.ylim([0.4, 1.2])
plt.ylabel('Disparate Impact (DI)')
plt.xlabel('repair level')
plt.show()

    References:
        .. [1] M. Feldman, S. A. Friedler, J. Moeller, C. Scheidegger, and
           S. Venkatasubramanian, "Certifying and removing disparate impact."
           ACM SIGKDD International Conference on Knowledge Discovery and Data
           Mining, 2015.